In [1]:
import sys
sys.path.append("./sktnb_code_challenge/ttuckbaegi3/metrics")
import pandas as pd

In [2]:
df = pd.read_csv('movie_list_label.tsv',sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393187 entries, 0 to 393186
Data columns (total 2 columns):
USER_ID       393187 non-null int64
movie_list    393187 non-null object
dtypes: int64(1), object(1)
memory usage: 6.0+ MB


In [3]:
df.head()

,USER_ID,movie_list
0,97140,"[2085, 3754, 1057, 1068, 2034, 8032, 3851, 265..."
1,176329,"[4155, 1720, 6561, 7441, 5026, 3262, 29, 7693,..."
2,184938,"[7382, 1520, 3705, 6024, 8084, 2420, 8158, 200..."
3,162887,"[3477, 829, 3549, 4007, 5354, 5199, 2537, 2658..."
4,327858,"[845, 6428, 4820, 28, 8075, 3987, 1697, 364, 6..."


In [4]:
def str2list(x):
    return list(map(int, x.replace('[','')\
                          .replace(']','')\
                          .replace(' ','')\
                          .split(',')))

In [5]:
str2list(df.iloc[0]['movie_list'])

[2085,
 3754,
 1057,
 1068,
 2034,
 8032,
 3851,
 2658,
 2895,
 1566,
 5533,
 1559,
 1032,
 3956,
 4949,
 845,
 1371,
 3987,
 6290,
 1645,
 5026,
 3574,
 5382,
 7864,
 1400,
 7824,
 5775,
 1330,
 2119,
 2229,
 6411]

In [6]:
%%time
df['movie_list'] = df['movie_list'].apply(str2list)
df['movie_list_len'] = df['movie_list'].apply(len)

CPU times: user 2.44 s, sys: 96 ms, total: 2.54 s
Wall time: 2.54 s


In [7]:
df.head()

,USER_ID,movie_list,movie_list_len
0,97140,"[2085, 3754, 1057, 1068, 2034, 8032, 3851, 265...",31
1,176329,"[4155, 1720, 6561, 7441, 5026, 3262, 29, 7693,...",28
2,184938,"[7382, 1520, 3705, 6024, 8084, 2420, 8158, 200...",31
3,162887,"[3477, 829, 3549, 4007, 5354, 5199, 2537, 2658...",26
4,327858,"[845, 6428, 4820, 28, 8075, 3987, 1697, 364, 6...",26


In [8]:
user_hist = pd.read_csv('./KISA_TBC_VIEWS.csv_team11')
user_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15823019 entries, 0 to 15823018
Data columns (total 5 columns):
USER_ID      int64
MOVIE_ID     int64
DURATION     int64
WATCH_DAY    int64
WATCH_SEQ    int64
dtypes: int64(5)
memory usage: 603.6 MB


### user_id 기반 k-fold

In [9]:
import time

In [10]:
from sklearn.model_selection import train_test_split
from average_precision import mapk

In [12]:
%%time

k = 5
results = []
for i in range(k):
    train, test = train_test_split(df,test_size=0.2, random_state=int(time.time()))
    train_user_hist = user_hist.merge(train, left_on='USER_ID', right_on='USER_ID', how='inner')
    answer = train_user_hist['MOVIE_ID'].value_counts()[:50].index.tolist()
    
    actual = test['movie_list'].tolist()
    predicted = [answer]*len(test)
    
    mapk50 = mapk(actual, predicted,k=50)
    results.append(mapk50)
    print(i, mapk50)
    
print('mean:',np.mean(results))

0 0.137082397249
1 0.136458182483
2 0.137659730934
3 0.136775422347
4 0.136086469884
mean: 0.136812440579
CPU times: user 26.1 s, sys: 3.87 s, total: 30 s
Wall time: 30 s
